# Faiss: A library for efficient similarity search and clustering of dense vectors 
- [Faiss-코드](https://github.com/facebookresearch/faiss)
- [Faiss-문서](https://github.com/facebookresearch/faiss/wiki)

## 0. 라이브러리 설치
- `faiss-cpu`와 `faiss-gpu`의 차이점
| 항목 | `faiss-cpu` | `faiss-gpu` |
| --- | --- | --- |
| 방법 | 벡터 인덱스 **CPU-RAM** 적재 | 벡터 인덱스 **GPU 메모리(VRAM)** 적재) |
| 연산 | CPU 기반 연산 | CUDA(GPU) 기반 연산 |
| 속도 | 느림 | 빠름 |
| 구현 | `IndexFlatL2` 사용 | faiss.index_cpu_to_gpu() 사용 |

In [1]:
!pip install pypdf # pypdf
!pip install spacy # spaCy
!pip install faiss-cpu # Faiss CPU 버전
# !pip install faiss-gpu # Faiss GPU 버전
!pip install langchain # LangChain Official
!pip install langchain-openai # LangChain Open-AI
!pip install langchain-community # LangChain Community

## 1. 패키지 로드

In [2]:
import os
import faiss
import getpass
import numpy as np
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import SpacyTextSplitter

## 2. OpenAI API Key 설정

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

Enter API key for OpenAI:  ········


## 3. Embedding Model 로드
| Model                   | ~ Pages per dollar | Performance on MTEB eval | Max input |
|------------------------|--------------------|----------------------------|-----------|
| text-embedding-3-small | 62,500             | 62.3%                      | 8192      |
| text-embedding-3-large | 9,615              | 64.6%                      | 8192      |
| text-embedding-ada-002 | 12,500             | 61.0%                      | 8192      |


In [4]:
dims = 1024
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", dimensions=dims)

## 4. Text Splitter Model 로드
**spaCy 한국어 지원 모델**
| 모델 이름               | 크기   | 구성 요소                                                                                     | 벡터 사용       | 토크나이저           |
|------------------------|--------|--------------------------------------------------------------------------------------------|------------------|----------------------|
| ko_core_news_sm        | sm     | tok2vec, tagger, morphologizer, parser, trainable_lemmatizer, senter, ner                | 없음             | rule-based (기본) :contentReference[oaicite:1]{index=1} |
| ko_core_news_md        | md     | tok2vec, tagger, morphologizer, parser, trainable_lemmatizer, senter, ner                | floret 벡터 (50k) | rule-based (기본) :contentReference[oaicite:2]{index=2} |
| ko_core_news_lg        | lg     | tok2vec, tagger, morphologizer, parser, trainable_lemmatizer, senter, ner                | floret 벡터 (200k)| rule-based (기본) :contentReference[oaicite:3]{index=3} |
| ko_core_news_trf       | trf    | transformer, tagger, parser, trainable_lemmatizer, senter, ner                           | transformer 기반 | rule-based (기본) :contentReference[oaicite:4]{index=4} |


In [5]:
# spaCy Text Splitter 모델 다운로드
# ko_core_news_lg -> 원하는 모델로 변경 가능
!python -m spacy download ko_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 MB 29.6 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_lg')


In [6]:
# spaCy Text Splitter 로드
text_splitter_model = SpacyTextSplitter(
    pipeline="ko_core_news_lg",
    chunk_size=300, # 최대 청크 크기
    chunk_overlap=50 # 청크 간 중복 허용 크기
)

## 4. 데이터 로드

In [7]:
pdf_loader = PyPDFLoader("../../data/Advancements and Applications of Artificial Intelligence Technologies Use Cases and Ethical Challenges_ko.pdf")
pdf_docs = pdf_loader.load()
pdf_docs[0].page_content[:100]

'인공지능은 인간의 사고 과정을 모방하여 문제를 해결하고, 학습하며, 추론할 수 있는 기술을 말한다. 특히 기계학습과 딥러닝 기술의 발전으로 인해 자연어 처리, 컴퓨터 비전, 음성 '

## 5. 청킹(Chunking)

In [8]:
documents = []
for i in range(len(pdf_docs)):
    # 텍스트 데이터
    content = pdf_docs[i].page_content

    # 메타데이터
    metadata = {"pageNum" : i+1, "title" : "Advancements and Applications of Artificial Intelligence Technologies Use Cases and Ethical Challenges_ko.pdf"}
    
    chunks = text_splitter_model.split_text(content)
    for chunk in chunks:
        documents.append(
            {"text" : chunk, "metadata" : metadata}
        )

In [9]:
documents

[{'text': "인공지능은 인간의 사고 과정을 모방하여 문제를 해결하고, 학습하며, 추론할 수 있는 기술을 말한다.\n\n특히 기계학습과 딥러닝 기술의 발전으로 인해 자연어 처리, 컴퓨터 비전, 음성 인식 등 다양한 분야에서 큰 성과를 거두고 있다.자연어 처리 분야에서는 대규모 언어 모델이 등장하며 기계 번역, 질의응답, 문서 요약 등에서 인간 수준에 근접한 성능을 보이고 있다.\n\n대표적인 모델로는 OpenAI의 GPT 시리즈, Google's BERT, Meta의 LLaMA 등이 있다.",
  'metadata': {'pageNum': 1,
   'title': 'Advancements and Applications of Artificial Intelligence Technologies Use Cases and Ethical Challenges_ko.pdf'}},
 {'text': '이러한 모델들은 사전학습(pretraining)과 미세조정(fine-tuning) 과정을 통해 특정 작업에 최적화된다.컴퓨터 비전 분야에서는 CNN(합성곱 신경망) 구조를 기반으로 이미지 분류, 객체 탐지, 세그멘테이션 등의 작업이 자동화되고 있다.\n\n최근에는 Vision Transformer(ViT)와 같은 트랜스포머 기반 모델도 활발히 연구되고 있다.',
  'metadata': {'pageNum': 1,
   'title': 'Advancements and Applications of Artificial Intelligence Technologies Use Cases and Ethical Challenges_ko.pdf'}},
 {'text': '자율주행차, 의료 영상 분석, 보안 감시 시스템 등에 실질적으로 적용되고 있다.음성 인식 기술은 스마트폰의 음성 비서, 고객센터의 AI 상담원, 그리고 회의 녹취 자동화 등에 활용되고 있다.\n\n이 기술은 파형 데이터를 스펙트로그램으로 변환하고 이를 딥러닝 모델로 해석하는 방식으로 이루어진다.\n\nWhisper나 De

## 6. 텍스트 임베딩(Text Embedding)

In [10]:
# 임베딩
embeddings = embedding_model.embed_documents([doc["text"] for doc in documents])

# Numpy float32 변환 (Faiss 입력 형식)
embeddings = np.array(embeddings).astype('float32')

embeddings[0][:10]

array([ 0.0006396 ,  0.01123521, -0.03570067, -0.00485028, -0.00958749,
        0.02888362,  0.00586396,  0.01158252,  0.0110979 , -0.03696069],
      dtype=float32)

## 7. 벡터 데이터베이스(Vector DB)

In [11]:
# Faiss 인덱스 생성
index = faiss.IndexFlatIP(dims)

In [12]:
# 벡터 정규화
faiss.normalize_L2(embeddings)
index.add(embeddings)

In [13]:
# 메타데이터 매핑
metadatas = {i : doc for i, doc in enumerate(documents)}

## 8. 시멘틱 검색

In [14]:
# 쿼리
query = "언어를 이해하는 인공지능 기술이란?"

In [15]:
# 쿼리 임베딩
query_embedding = embedding_model.embed_documents([query])
query_embedding = np.array(query_embedding).astype("float32")

In [16]:
# 벡터 정규화
faiss.normalize_L2(query_embedding)

In [17]:
# 유사한 벡터 검색 (상위 3개)
D, I = index.search(query_embedding, k=3)

In [18]:
# 결과 출력
print("검색 결과:")
for rank, idx in enumerate(I[0]):
    result  = metadatas[idx]
    print(f"{rank+1}. 텍스트: {result['text']} / 메타데이터: {result['metadata']}")
    print("=" * 50)

검색 결과:
1. 텍스트: 인공지능은 인간의 사고 과정을 모방하여 문제를 해결하고, 학습하며, 추론할 수 있는 기술을 말한다.

특히 기계학습과 딥러닝 기술의 발전으로 인해 자연어 처리, 컴퓨터 비전, 음성 인식 등 다양한 분야에서 큰 성과를 거두고 있다.자연어 처리 분야에서는 대규모 언어 모델이 등장하며 기계 번역, 질의응답, 문서 요약 등에서 인간 수준에 근접한 성능을 보이고 있다.

대표적인 모델로는 OpenAI의 GPT 시리즈, Google's BERT, Meta의 LLaMA 등이 있다. / 메타데이터: {'pageNum': 1, 'title': 'Advancements and Applications of Artificial Intelligence Technologies Use Cases and Ethical Challenges_ko.pdf'}
2. 텍스트: 예를 들어, 텍스트와 이미지를 동시에 처리하는 모델은 이미지 캡셔닝, 시각 질의응답(VQA), 문서 이해 등에 활용된다.

CLIP, Flamingo, Kosmos 등의 모델이 대표적이다.한편, 이러한 인공지능 기술의 확산에 따라 윤리적 이슈와 프라이버시 보호 문제도 중요하게 대두되고 있다.

알고리즘의 편향성, 투명성, 설명 가능성(XAI), 개인정보 보호 등은 앞으로 인공지능 시스템이 실생활에 널리 적용되기 위한 중요한 과제들이다. / 메타데이터: {'pageNum': 1, 'title': 'Advancements and Applications of Artificial Intelligence Technologies Use Cases and Ethical Challenges_ko.pdf'}
3. 텍스트: 자율주행차, 의료 영상 분석, 보안 감시 시스템 등에 실질적으로 적용되고 있다.음성 인식 기술은 스마트폰의 음성 비서, 고객센터의 AI 상담원, 그리고 회의 녹취 자동화 등에 활용되고 있다.

이 기술은 파형 데이터를 스펙트로그램으로 변환하고 이를 딥러닝 모델로 해석하는 방식으로 이루어진다.

W